## Dataset preaparation

#### Import necessary packages

In [632]:
import numpy as np
import pandas as pd

#### Import Data form csv

In [633]:
# Define the paths to the files to import
text='/home/antonio/code/pollypineapple1/FED-Predictor/raw_data/Fed_Scrape-2015-2023.csv'
rates='/home/antonio/code/pollypineapple1/FED-Predictor/raw_data/US Fed Rate.csv'

In [634]:
# Import files
text_df=pd.read_csv(text)
rates_df=pd.read_csv(rates)

#### Check the data

In [635]:
text_df.head()

,Unnamed: 0,Date,Type,Text
0,0,20230412,0,The Federal Reserve on Wednesday released the ...
1,1,20230412,0,The minutes for each regularly scheduled meeti...
2,2,20230412,0,The minutes can be viewed on the Board's website.
3,3,20230412,0,"For media inquiries, e-mail [email protected] ..."
4,4,20230412,0,Minutes of the Federal Open Market Committee\r...


In [636]:
rates_df.head()

,Release Date,Time,Actual,Forecast,Previous
0,"Nov 01, 2023",13:00,5.50%,5.50%,5.50%
1,"Sep 20, 2023",13:00,5.50%,5.50%,5.50%
2,"Jul 26, 2023",13:00,5.50%,5.50%,5.25%
3,"Jun 14, 2023",13:00,5.25%,5.25%,5.25%
4,"May 03, 2023",13:00,5.25%,5.25%,5.00%


In [637]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  9974 non-null   int64 
 1   Date        9974 non-null   int64 
 2   Type        9974 non-null   int64 
 3   Text        9974 non-null   object
dtypes: int64(3), object(1)
memory usage: 311.8+ KB


In [638]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Release Date  201 non-null    object
 1   Time          201 non-null    object
 2   Actual        201 non-null    object
 3   Forecast      201 non-null    object
 4   Previous      201 non-null    object
dtypes: object(5)
memory usage: 8.0+ KB


#### Remove useless columns and format relevant ones

In [639]:
columns_to_keep = ['Release Date', 'Actual']
rates_df=rates_df[columns_to_keep]
rates_df=rates_df.rename(columns = {'Release Date' : 'date'})

In [640]:
rates_df['date'] = rates_df['date'].astype(str)

In [641]:
rates_df.head()

,date,Actual
0,"Nov 01, 2023",5.50%
1,"Sep 20, 2023",5.50%
2,"Jul 26, 2023",5.50%
3,"Jun 14, 2023",5.25%
4,"May 03, 2023",5.25%


Conversion to datetime

In [642]:
rates_df['date'] = pd.to_datetime(rates_df['date'], format = '%b %d, %Y')

In [643]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    201 non-null    datetime64[ns]
 1   Actual  201 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 3.3+ KB


In [644]:
rates_df=rates_df.rename(columns={'Actual': 'rate'})

In [645]:
# ensure rates are number object
rates_df['rate']=rates_df['rate'].str.strip('%').astype(float)

In [646]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    201 non-null    datetime64[ns]
 1   rate    201 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.3 KB


In [647]:
rates_df.head()

,date,rate
0,2023-11-01,5.50
1,2023-09-20,5.50
2,2023-07-26,5.50
3,2023-06-14,5.25
4,2023-05-03,5.25


In [648]:
text_df['Date']=pd.to_datetime(text_df['Date'], format='%Y%m%d')
text_df=text_df.rename(columns={'Date': 'date'})
text_df=text_df.rename(columns={'Text': 'text'})
text_df=text_df.rename(columns={'Type': 'type'})

In [649]:
text_df.drop(['Unnamed: 0'], axis=1, inplace=True)

#### Convert type to text

In [650]:
# convert classification to number
text_df['type_text'] = text_df['type'].apply(lambda x: 'statement' if x == 0 else 'minutes')

In [651]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       9974 non-null   datetime64[ns]
 1   type       9974 non-null   int64         
 2   text       9974 non-null   object        
 3   type_text  9974 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 311.8+ KB


#### Convert rates to text

In [652]:
# convert numbers into up or down
rates_df['rate_change'] = rates_df['rate'].diff()
rates_df['rate_change_text'] = rates_df['rate_change'].apply(lambda x: 'up' if x > 0 else ('down' if x < 0 else 'no change')).astype(str)

In [653]:
rates_df.head()

,date,rate,rate_change,rate_change_text
0,2023-11-01,5.50,NaN,no change
1,2023-09-20,5.50,0.00,no change
2,2023-07-26,5.50,0.00,no change
3,2023-06-14,5.25,-0.25,down
4,2023-05-03,5.25,0.00,no change


#### Merge the two dataframes based on Date

Sort by dates

In [654]:
sorted_rates=rates_df.sort_values(by='date')
sorted_text=text_df.sort_values(by='date')

In [655]:
sorted_rates.head()

,date,rate,rate_change,rate_change_text
200,1990-07-13,8.00,0.25,up
199,1990-10-29,7.75,0.25,up
198,1990-11-14,7.50,0.25,up
197,1990-12-07,7.25,0.25,up
196,1990-12-19,7.00,0.25,up


In [656]:
sorted_text.head()

,date,type,text,type_text
9973,2015-01-07,0,"\r\n For media inquiries, call 202-452-2...",statement
9969,2015-01-07,0,\r\n The Federal Reserve Board and the F...,statement
9972,2015-01-07,0,\nMinutes of the Federal Open Market Committee...,statement
9971,2015-01-07,0,\r\n FOMC minutes can be viewed on the B...,statement
9970,2015-01-07,0,\r\n The minutes for each regularly sche...,statement


Merge based on date. >> Direction backward << to ensure rates decisions are paired with previous text

In [657]:
df_merged = pd.merge_asof(sorted_text, sorted_rates, on='date')

In [658]:
df_merged.head()

,date,type,text,type_text,rate,rate_change,rate_change_text
0,2015-01-07,0,"\r\n For media inquiries, call 202-452-2...",statement,0.25,0.0,no change
1,2015-01-07,0,\r\n The Federal Reserve Board and the F...,statement,0.25,0.0,no change
2,2015-01-07,0,\nMinutes of the Federal Open Market Committee...,statement,0.25,0.0,no change
3,2015-01-07,0,\r\n FOMC minutes can be viewed on the B...,statement,0.25,0.0,no change
4,2015-01-07,0,\r\n The minutes for each regularly sche...,statement,0.25,0.0,no change


In [659]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              9974 non-null   datetime64[ns]
 1   type              9974 non-null   int64         
 2   text              9974 non-null   object        
 3   type_text         9974 non-null   object        
 4   rate              9974 non-null   float64       
 5   rate_change       9974 non-null   float64       
 6   rate_change_text  9974 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 545.6+ KB


In [660]:
columns_to_keep = ['date', 'type_text', 'text','rate_change_text']
df_merged=df_merged[columns_to_keep]

In [661]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              9974 non-null   datetime64[ns]
 1   type_text         9974 non-null   object        
 2   text              9974 non-null   object        
 3   rate_change_text  9974 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 311.8+ KB


## Data Preprocessing

#### Import necessary packages

In [662]:
import string
import re

import nltk
#punkt_tab is needed over punkt because of working with tabular data (dataframe)
#nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import OrdinalEncoder

### Text Variables Preocessing

#### Cleaning

In [663]:
# Remove all the whitespaces at the beginning and the end of a string
# strip()
df_cleaned = df_merged.copy()
df_cleaned['text']=df_merged['text'].str.strip()

In [664]:
# Text modeling algorithms are case-sensitive
# lower()
df_cleaned['text']=df_cleaned['text'].str.lower()

In [665]:
# Remove numbers and symbols to ensure better text clustering and keyphrases collection
# replace(r'[^a-zA-Z\s]')
# a-zA-Z: Matches all lowercase (a-z) and uppercase (A-Z) alphabetic characters
# \s: Matches any whitespace character (space, tab, newline)
# [^...]: Matches anything NOT included in the brackets. This removes everything that isn't a letter or whitespace
df_cleaned['text'] = df_cleaned['text'].str.replace(r'[^a-zA-Z\s]', "", regex=True)

In [666]:
df_cleaned['text'].head(20)

0                             for media inquiries call 
1     the federal reserve board and the federal open...
2     minutes of the federal open market committee\r...
3     fomc minutes can be viewed on the boards websi...
4     the minutes for each regularly scheduled meeti...
5                             for media inquiries call 
6                              the minutes are attached
7     the federal reserve board on tuesday released ...
8     iv subject to reasonable limitations on the am...
9                               b the selected bank may
10    i reject bids that as determined in its sole d...
11    ii accept treasury securities or cash as colla...
12    iii accept agency securities as collateral onl...
13    a the selected bank for the soma to undertake ...
14    iii subject to reasonable limitations on the t...
15    b any federal reserve bank that maintains cust...
16    i undertake repo transactions in eligible secu...
17    ii undertake intraday reverse repo transac

#### Tokenizing

In [667]:
df_tokenized=df_cleaned.copy()

In [668]:
df_tokenized['text']=df_tokenized['text'].apply(word_tokenize)

In [669]:
df_tokenized['text'].head()

0                        [for, media, inquiries, call]
1    [the, federal, reserve, board, and, the, feder...
2    [minutes, of, the, federal, open, market, comm...
3    [fomc, minutes, can, be, viewed, on, the, boar...
4    [the, minutes, for, each, regularly, scheduled...
Name: text, dtype: object

#### Stopwords

### Do i remove stopwords in this case?

In [670]:
# removes defined stopwords
#   uses nltk.corpus to define stopwords
#   iterates over each word (alredy tokenized) to filter out stopwords

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word.lower() not in stop_words]

In [671]:
df_stopwords = df_tokenized.copy()

In [672]:
df_stopwords['text'] = df_stopwords['text'].apply(remove_stopwords)

In [673]:
# Check for lenght differences
tokenized_lenght = df_tokenized['text'].apply(len)
stopwords_lenght = df_stopwords['text'].apply(len)

print('Tokenized Lenght: ', tokenized_lenght.sum())
print('Stopwords Lenght: ', stopwords_lenght.sum())

Tokenized Lenght:  647423
Stopwords Lenght:  408075


#### Lemmatizing

In [674]:
# lemmatizes tokenized words in a list
#   uses nltk.corpus to define stopwords
#   iterates over each word (alredy tokenized) to filter out stopwords

def lemmatize_tokens(tokens):
    lemmatizer = WordNetLemmatizer()
    return [WordNetLemmatizer().lemmatize(word) for word in tokens]

In [675]:
df_lemmatized=df_tokenized.copy()
df_lemmatized['text'] = df_lemmatized['text'].apply(lemmatize_tokens)

In [676]:
stopwords_lenght = df_tokenized['text'].apply(len)
lemmatized_length = df_lemmatized['text'].apply(len)

print('Original Lenght: ', stopwords_lenght.sum())
print('Lemmatized Lenght: ', lemmatized_length.sum())

Original Lenght:  647423
Lemmatized Lenght:  647423


In [677]:
# Check if lemmatization actually changed any tokens by comparing original and lemmatized text for the first row
print(f"Original Tokens (First Row): {df_tokenized['text'].iloc[0]}")
print(f"Lemmatized Tokens (First Row): {df_lemmatized['text'].iloc[0]}")

Original Tokens (First Row): ['for', 'media', 'inquiries', 'call']
Lemmatized Tokens (First Row): ['for', 'medium', 'inquiry', 'call']


### Other Variables Processing

#### One-hot-encode

In [678]:
df_lemmatized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              9974 non-null   datetime64[ns]
 1   type_text         9974 non-null   object        
 2   text              9974 non-null   object        
 3   rate_change_text  9974 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 311.8+ KB


In [679]:
print(df_lemmatized['rate_change_text'].value_counts())
print(df_lemmatized['type_text'].value_counts())

rate_change_text
no change    6353
down         2952
up            669
Name: count, dtype: int64
type_text
minutes      8245
statement    1729
Name: count, dtype: int64


In [680]:
ordinal_encoder = OrdinalEncoder()

In [681]:
ordinal_encoder.fit(df_lemmatized[["rate_change_text"]])
display(ordinal_encoder.categories_)

[array(['down', 'no change', 'up'], dtype=object)]

In [682]:
df_lemmatized["rate_decision_encoded"] = ordinal_encoder.transform(df_lemmatized[["rate_change_text"]]).astype(int)

In [683]:
print(df_lemmatized['rate_decision_encoded'].value_counts())

rate_decision_encoded
1    6353
0    2952
2     669
Name: count, dtype: int64


In [684]:
ordinal_encoder.fit(df_lemmatized[["type_text"]])
display(ordinal_encoder.categories_)

[array(['minutes', 'statement'], dtype=object)]

In [685]:
df_lemmatized['type_encoded'] = ordinal_encoder.transform(df_lemmatized[['type_text']]).astype(int)

In [686]:
print(df_lemmatized['type_encoded'].value_counts())

type_encoded
0    8245
1    1729
Name: count, dtype: int64


In [687]:
df_lemmatized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   9974 non-null   datetime64[ns]
 1   type_text              9974 non-null   object        
 2   text                   9974 non-null   object        
 3   rate_change_text       9974 non-null   object        
 4   rate_decision_encoded  9974 non-null   int64         
 5   type_encoded           9974 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 467.7+ KB


In [688]:
columns_to_keep = ['date', 'type_encoded', 'text', 'rate_decision_encoded']
df_encoded=df_lemmatized[columns_to_keep]

In [689]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   9974 non-null   datetime64[ns]
 1   type_encoded           9974 non-null   int64         
 2   text                   9974 non-null   object        
 3   rate_decision_encoded  9974 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 311.8+ KB


In [690]:
df_encoded['text'].head()

0                         [for, medium, inquiry, call]
1    [the, federal, reserve, board, and, the, feder...
2    [minute, of, the, federal, open, market, commi...
3    [fomc, minute, can, be, viewed, on, the, board...
4    [the, minute, for, each, regularly, scheduled,...
Name: text, dtype: object

In [691]:
# model of chocie requires input as single string, not tokenized)
df_final=df_encoded
df_final['text']=df_encoded['text'].apply(lambda x: ' '.join(x))

/tmp/ipykernel_4769/1241547755.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['text']=df_encoded['text'].apply(lambda x: ' '.join(x))


In [692]:
df_encoded['text'].head()

0                              for medium inquiry call
1    the federal reserve board and the federal open...
2    minute of the federal open market committee de...
3    fomc minute can be viewed on the board website...
4    the minute for each regularly scheduled meetin...
Name: text, dtype: object

## Baseline Model

In [708]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score

### Train test split

In [694]:
X_train, X_test, y_train, y_test = train_test_split(df_final['text'], df_final['rate_decision_encoded'], test_size=0.3, random_state=42)

In [695]:
X_train.head()

5991    with respect to the business sector participan...
5860    ii undertake intraday repo transaction in elig...
6022    at the conclusion of the discussion the commit...
9061    with respect to the business sector participan...
5156                                                     
Name: text, dtype: object

In [696]:
y_train.head()

5991    2
5860    2
6022    2
9061    0
5156    1
Name: rate_decision_encoded, dtype: int64

### Pipeline: vectorization and modeling

In [697]:
# converting raw text into a numerical representation
# TfidfVectorizer is the vectorizer of choice. It takes into account the importance of words within the corpus
# Naive Bayes Algorithm is the algorithm of choice. It's simple and quick. Can provide an adequate base score

tf_idf_vectorizer = TfidfVectorizer()

model = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [698]:
model.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [710]:
# unbalanced classes. need to 
accuracy = model.score(X_test, y_test)
print("Model Accuracy:", accuracy)

Model Accuracy: 0.6538590043434681


In [711]:
y_pred = model.predict(X_test)

In [712]:
# unbalanced classes. better use f1-weighted
f1_weighted = f1_score(y_test, y_pred, average='weighted')
print(f"F1-weighted score: {f1_weighted}")

F1-weighted score: 0.5377960016406653


### Cross Validation

In [713]:
# classes are heavily unbalanced. there are only 669 occurrences of up so f1_weighted is better for GS
# this computes the F1-score for each class, but it will weight each class's score by its frequency in the data
# AKA, more forgiving towards 'up' occurrences

param_grid = {
    'tfidfvectorizer__ngram_range': [(1, 1), (1, 2)],  # Unigrams or bigrams
    'multinomialnb__alpha': [0.1, 0.5, 1.0]  # Alpha values for smoothing
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_weighted')

In [706]:
grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best F1 score (macro): {grid_search.best_score_}")

Best parameters: {'multinomialnb__alpha': 0.1, 'tfidfvectorizer__ngram_range': (1, 2)}
Best F1 score (macro): 0.602634107928488
